In [2]:
import os
import configparser
import oracledb
import pandas as pd
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s — %(name)s — %(levelname)s — %(message)s')
logger = logging.getLogger(__name__)

def get_db_config(env, config_path='config/config.ini'):
    """Reads the database configuration for the given environment."""
    config = configparser.ConfigParser()
    config.read(config_path)
    if env in config:
        return config[env]
    else:
        raise ValueError(f"Configuration for the environment '{env}' not found.")

def connect_to_database(config):
    """Connects to the Oracle database using the provided configuration."""
    return oracledb.connect(user=config['username'], password=config['password'], dsn=config['dsn'])

def fetch_data(query, connection):
    """Fetches data from the database using the given connection and query."""
    with connection.cursor() as cursor:
        cursor.execute(query)
        columns = [col[0] for col in cursor.description]
        return pd.DataFrame(cursor.fetchall(), columns=columns)

def main():
    environment = os.getenv('ENVIRONMENT', 'production_app')  # Default to 'test' if not set
    config_path = os.getenv('CONFIG_PATH', 'config/config.ini')
    
    try:
        db_config = get_db_config(environment, config_path)
        
        # Initialize connection as None
        conn = None

        # Connect to the database
        conn = connect_to_database(db_config)
        logger.info(f"Connected to the {environment} database successfully.")
        
        query = 'SELECT * FROM CUSTOMERS'
        df = fetch_data(query, conn)
        
        # Display DataFrame
        with pd.option_context('display.max_rows', None, 'display.max_columns', None):
            print(df)
    
    except oracledb.DatabaseError as e:
        error, = e.args
        logger.error(f"Database error occurred: {error.code}, {error.message}")
    
    except Exception as e:
        logger.error(f"An unexpected error occurred: {e}")

    finally:
        # Close the database connection if it was established
        if conn is not None:
            try:
                conn.close()
                logger.info(f"Database connection to {environment} closed.")
            except Exception as e:
                logger.error(f"Failed to close the database connection: {e}")

if __name__ == '__main__':
    main()


2024-08-30 10:35:31,858 — __main__ — INFO — Connected to the production_app database successfully.
2024-08-30 10:35:31,876 — __main__ — INFO — Database connection to production_app closed.


     CUSTOMER_ID                      EMAIL_ADDRESS              FULL_NAME
0            287        kristina.nunez@internalmail         Kristina Nunez
1            288         mable.ballard@internalmail          Mable Ballard
2            289       diane.wilkerson@internalmail        Diane Wilkerson
3            290          sheryl.banks@internalmail           Sheryl Banks
4            291             opal.cruz@internalmail              Opal Cruz
5            292           dale.hughes@internalmail            Dale Hughes
6            293          diana.fowler@internalmail           Diana Fowler
7            294       travis.schwartz@internalmail        Travis Schwartz
8            295         anthony.boone@internalmail          Anthony Boone
9            296            tim.harmon@internalmail             Tim Harmon
10           297        kristin.brooks@internalmail         Kristin Brooks
11           298          kathy.dennis@internalmail           Kathy Dennis
12           299         